In [2]:
import os

In [3]:
# Define the path to the data folder
data_folder = '../data/'

In [4]:
# Load one of the downloaded books
file_path = os.path.join(data_folder, 'pg1342.txt')

In [5]:
# Read the content of the file
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

In [ ]:
# Display the first 1000 characters
print(text[:1000])

In [9]:
# Identify the start and end of the main text
start_marker = '*** Start of Project'
end_marker = '*** End of Project'

# Find the positions of the markers
start_pos = text.find(start_marker)
end_pos = text.find(end_marker)

# Display the positions of the start and end markers
print(f"Start marker found at: {start_pos}")
print(f"End marker found at: {end_pos}")


Start marker found at: -1
End marker found at: -1
